### Segmented Customers based on RFM SCORES (Recency, Frequency, Monetary) with the help of RFM analysis

In [22]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [23]:
#reading data
data = pd.read_csv('retail.csv', encoding = 'unicode_escape')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12-09-2011,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12-09-2011,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12-09-2011,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12-09-2011,4.15,12680.0,France


In [24]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


# EDA

In [25]:
#converting invoice date to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [26]:
#excluding negative values
data = data[(data.Quantity >0) & (data.UnitPrice >0)]

In [27]:
#Returned items are indicated with C
data = data[~data['StockCode'].str.contains('C')]

In [28]:
#dropping duplicates
data = data.drop_duplicates()

In [29]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,518463.000000,518463.000000,388538.000000
mean,10.653740,3.917787,15286.857556
std,157.225075,36.305861,1714.140334
min,1.000000,0.001000,12346.000000
25%,1.000000,1.250000,13952.000000
50%,4.000000,2.080000,15150.000000
75%,12.000000,4.130000,16791.000000
max,80995.000000,13541.330000,18287.000000


In [30]:
data.dropna(inplace=True)

In [32]:
#Total price column
data['Total_Price'] = data['UnitPrice']*data['Quantity']

In [33]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


In [35]:
data['InvoiceDate'].max()

Timestamp('2011-12-09 00:00:00')

In [36]:
now =  dt.datetime(2011,12,10)

## RFM DATAFRAME

In [39]:
rfm = data.groupby('CustomerID').agg({'InvoiceDate' : lambda day : (now - day.max()).days,
                               'InvoiceNo': lambda num : len(num),
                              'Total_Price': lambda price : price.sum()
                             
                             
                             })

cols = ['Recency','Frequency','Monetary']

In [43]:
rfm.columns = cols

# Calculating RFM scores

In [44]:
rfm["R"] = pd.qcut(rfm["Recency"],5,labels=[5,4,3,2,1])

In [45]:
rfm["F"] = pd.qcut(rfm["Frequency"],5,labels=[1,2,3,4,5])
rfm["M"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm["RFM_Score"] = rfm["R"].astype(str) +rfm["F"].astype(str) + rfm["M"].astype(str)

In [46]:
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score
CustomerID,,,,,,,
12346.0,325,1,77183.60,1,1,5,115
12347.0,3,179,4256.90,5,5,5,555
12348.0,75,31,1797.24,2,3,4,234
12349.0,18,73,1757.55,4,4,4,444
12350.0,311,15,306.50,1,2,2,122


In [47]:
rfm[rfm.RFM_Score == '111'].sort_values(by='Monetary',ascending=False)

,Recency,Frequency,Monetary,R,F,M,RFM_Score
CustomerID,,,,,,,
17925.0,373,1,244.08,1,1,1,111
14501.0,184,9,241.13,1,1,1,111
16427.0,263,9,241.06,1,1,1,111
17862.0,323,10,240.90,1,1,1,111
16756.0,215,13,239.40,1,1,1,111
...,...,...,...,...,...,...,...
13120.0,238,1,30.60,1,1,1,111
17102.0,261,1,25.50,1,1,1,111
17763.0,263,1,15.00,1,1,1,111


#### using recency and frequency to segment customers

In [48]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [49]:
rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Segment
CustomerID,,,,,,,,
12346.0,325,1,77183.60,1,1,5,115,Hibernating
12347.0,3,179,4256.90,5,5,5,555,Champions
12348.0,75,31,1797.24,2,3,4,234,At Risk
12349.0,18,73,1757.55,4,4,4,444,Loyal Customers
12350.0,311,15,306.50,1,2,2,122,Hibernating


In [50]:
rfm.groupby('Segment').mean().sort_values('Monetary')

,Recency,Frequency,Monetary
Segment,,,
Promising,23.148148,7.777778,418.106481
About to Sleep,52.582822,15.947853,448.272638
Hibernating,209.930018,13.659300,531.554725
Need Attention,52.406863,41.225490,821.920392
Potential Loyalists,16.068548,34.723790,897.880022
At Risk,163.893885,56.625899,1000.168022
Can't Loose,145.345679,177.493827,2379.549889
Loyal Customers,34.045512,153.356511,2813.549494
New Customers,7.634921,7.650794,3352.233333
